In [1]:
import pandas as pd
from transformers import pipeline
from datasets import load_dataset
# from tqdm.auto import tqdm
# import torch

In [9]:
# import pandas as pd
# df = pd.read_parquet('../data/5b.cryptonews_topic_sentiment.parquet')
# df

,title,text,source_name,date,topic,topic_confidence_score,news_text_title,sentiment_finbert,score_finbert,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa,People,Organizations,Cryptocurrencies,Events
0,2021 Bitcoin Price Predictions: Is The Massive...,As the bitcoin price hovers under the psycholo...,Forbes,2021-01-02 00:20:00+00:00,"liquidity, volatility, price action, price mov...",0.238574,2021 Bitcoin Price Predictions: Is The Massive...,neutral,0.876394,0.5,0.0,0.0,0.0,0.0,0.0,[],[],[Bitcoin],[]
1,Will Central Banks Hold Bitcoin in 2021?,Central banks (CB) will hold bitcoin sooner or...,BeInCrypto,2021-01-01 20:31:35+00:00,consensus updates,0.199005,Will Central Banks Hold Bitcoin in 2021? Centr...,neutral,0.916949,0.0,0.5,0.0,0.0,0.7,0.0,[Krüger],[Central banks],[Bitcoin],[]
2,"Bitcoin Is Digital Social Justice, feat. Tyron...",The podcaster and CEO of Onramp Invest discuss...,Coindesk,2021-01-01 19:15:02+00:00,social platforms,0.299585,"Bitcoin Is Digital Social Justice, feat. Tyron...",neutral,0.903567,0.0,0.3,0.0,0.5,0.7,0.0,[Tyrone Ross],[Onramp Invest],[Bitcoin],[]
3,Bitcoin hits all-time high against gold as hav...,"BTC has hit another milestone, this time again...",Cointelegraph,2021-01-01 18:52:00+00:00,"liquidity, volatility, price action, price mov...",0.430232,Bitcoin hits all-time high against gold as hav...,positive,0.931128,1.0,0.0,0.0,0.0,0.0,0.0,[],[],"[BTC, Bitcoin]",[]
4,"The Last Time This Indicator Flashed, Bitcoin ...",Bitcoin has been facing some turbulence as of ...,Bitcoinist,2021-01-01 18:00:00+00:00,"liquidity, volatility, price action, price mov...",0.522232,"The Last Time This Indicator Flashed, Bitcoin ...",negative,0.716469,0.3,0.0,0.0,0.0,0.0,0.0,[],[],"[Bitcoin, BTC]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155371,XRP News Today: BTC Hits $91k; Will SEC Appeal...,"XRP price drops below $2, weighed by SEC appea...",FXEmpire,2024-12-31 05:34:31+00:00,legal proceedings,0.407502,XRP News Today: BTC Hits $91k; Will SEC Appeal...,negative,0.908160,0.0,0.0,0.0,0.0,0.0,0.0,[],[SEC],"[XRP, BTC]",[]
155372,Tether Moves Bitcoin Worth $780M to Reserve in...,"Tether, the issuer of the world's most widely ...",Cryptonews,2024-12-31 05:28:54+00:00,"treasury, reserve",0.280455,Tether Moves Bitcoin Worth $780M to Reserve in...,neutral,0.741839,0.0,0.0,0.0,0.0,0.0,0.0,[],[Tether],[Bitcoin],[]
155373,The Bull Run Continues; Bitfinex Projects $200...,Digital asset exchange Bitfinex predicted an u...,Zycrypto,2024-12-31 05:21:59+00:00,market sentiment,0.487648,The Bull Run Continues; Bitfinex Projects $200...,positive,0.858072,0.0,0.0,0.0,0.0,0.0,0.0,[],[Bitfinex],"[Bitcoin, BTC]",[]
155374,MicroStrategy shares drop 8% after year-end Bi...,MicroStrategy shares plunged more than 8% foll...,Cryptopolitan,2024-12-31 05:08:01+00:00,market sentiment,0.321372,MicroStrategy shares drop 8% after year-end Bi...,negative,0.973286,0.0,0.0,0.0,0.0,0.0,0.0,[],[MicroStrategy],[Bitcoin],[]


### Approach 1: pretrained model - Jean-Baptiste/roberta-large-ner-english
- poor performance

In [3]:
dataset = load_dataset('parquet', data_files='../data/5b.cryptonews_topic_sentiment.parquet')['train']
print(dataset)

# select first 1000 rows
dataset = dataset.select(range(1000))

dataset = dataset.map(lambda example: {
  'news_text_title': example['title'] + ' ' + example['text']
})

Dataset({
    features: ['title', 'text', 'source_name', 'date', 'topic', 'topic_confidence_score', 'news_text_title', 'sentiment_finbert', 'score_finbert', 'tanalysis_absa', 'economy_absa', 'regulation_absa', 'technology_absa', 'adoption_absa', 'cybersecurity_absa', 'People', 'Organizations', 'Cryptocurrencies', 'Events'],
    num_rows: 155376
})


In [4]:
# test
# NER model from Hugging Face
pipe = pipeline("token-classification", model="Jean-Baptiste/roberta-large-ner-english")
txt = "Bank of America: Bitcoin Bull Run Is the 'Mother of All Bubbles' A senior analyst at Bank of America thinks that Bitcoin's latest bull run “blows the doors off prior bubbles."
ner_result = pipe(txt)

Device set to use cuda:0


In [5]:
# NER model from Hugging Face
pipe = pipeline("token-classification", model="Jean-Baptiste/roberta-large-ner-english")

def NER(batch):
  batch['NER'] = pipe(batch['news_text_title'])
  return batch

dataset = dataset.map(NER, batched=True, batch_size=5)

Device set to use cuda:0


In [6]:
def post_process_ner(ner_output):
  processed_entities = []
  current_entity = {}
  
  for token in ner_output:
    if current_entity and token['entity'] == current_entity['entity'] and token['start'] == current_entity['end'] + 1:
      current_entity['word'] += token['word'].replace('Ġ', ' ')
      current_entity['end'] = token['end']
    else:
      if current_entity:
        processed_entities.append(current_entity)
      current_entity = token.copy()
  
  if current_entity:
    processed_entities.append(current_entity)
  
  return processed_entities

def get_entities(processed_ner_output, entity_type=None):
  if entity_type is None:
    return {entity['word'][1:] for entity in processed_ner_output}
  else:
    return {entity['word'][1:] for entity in processed_ner_output if entity['entity'] == entity_type}


# aply post-processing and get entities from NER results
dataset = dataset.map(lambda batch: {
  'entities': get_entities(post_process_ner(batch['NER']))
})

In [7]:
df = dataset.to_pandas()
df[['title','text','entities']]

# poor performance, need to look for another NER approach

,title,text,entities
0,2021 Bitcoin Price Predictions: Is The Massive...,As the bitcoin price hovers under the psycholo...,[]
1,Will Central Banks Hold Bitcoin in 2021?,Central banks (CB) will hold bitcoin sooner or...,"[Kr, er, ¼]"
2,"Bitcoin Is Digital Social Justice, feat. Tyron...",The podcaster and CEO of Onramp Invest discuss...,"[, Tyr, i, On, ne Ross, mp Invest, De]"
3,Bitcoin hits all-time high against gold as hav...,"BTC has hit another milestone, this time again...",[BTC]
4,"The Last Time This Indicator Flashed, Bitcoin ...",Bitcoin has been facing some turbulence as of ...,"[BTC, Bitcoin]"
...,...,...,...
995,Associated Gas Pilot Could Lead to Huge Boon f...,Russian Bitcoin (BTC) miners could be in for a...,"[Russian, , ns, t, TC, Gaz, Kh, rom, Russian B..."
996,"Bitcoin Mining Revenue Hits $20 Billion, Doubl...",Bitcoin miners are making a killing from this ...,[]
997,Low-Fee Bitcoin Trust to Challenge Grayscale,Custodial Fee Wars Cryptocurrency is still bar...,"[, Bit, IT, B, ise 10 Crypto Index Fund, ys, a..."
998,Bitcoin Faced First Major Correction In Curren...,This was one of the most intense weeks. Bitcoi...,"[Update, The Crypto Weekly]"


### Approach 2: fine-tune a model


In [8]:
'''
Choose a Base Model
General-purpose: bert-base-uncased, roberta-base
Domain-specific: ElKulako/cryptobert (for crypto), ProsusAI/finbert (finance), or modernbert?

Load NER dataset (e.g., CoNLL-2003, or any other crypo news dataset with NER annotations)
Tokenize and Align Labels
Fine-tune the Model
Evaluate the Model (metrics like precision, recall, and F1-score)
'''

'\nChoose a Base Model\nGeneral-purpose: bert-base-uncased, roberta-base\nDomain-specific: ElKulako/cryptobert (for crypto), ProsusAI/finbert (finance), or modernbert?\n\nLoad NER dataset (e.g., CoNLL-2003, or any other crypo news dataset with NER annotations)\nTokenize and Align Labels\nFine-tune the Model\nEvaluate the Model (metrics like precision, recall, and F1-score)\n'